<a href="https://colab.research.google.com/github/sudhan670/curly-lab/blob/main/PythonStream.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pandas scikit-learn transformers torch streamlit


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [25]:
import pandas as pd
import streamlit as st
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from transformers import TrainingArguments, Trainer
import numpy as np


In [14]:
@st.cache
def load_data(file_path):
    return pd.read_csv(file_path)

data = load_data('data1.csv')
print(data)


2024-07-29 07:25:30.359 
`st.cache` is deprecated and will be removed soon. Please use one of Streamlit's new caching commands, `st.cache_data` or `st.cache_resource`.
More information [in our docs](https://docs.streamlit.io/develop/concepts/architecture/caching).

**Note**: The behavior of `st.cache` was updated in Streamlit 1.36 to the new caching logic used by `st.cache_data` and `st.cache_resource`.
This might lead to some problems or unexpected behavior in certain edge cases.

2024-07-29 07:25:30.362 No runtime found, using MemoryCacheStorageManager


                                       Query            Category
0               What is the meaning of life?  document_retrieval
1        Show me documents about blockchain.  document_retrieval
2             What is the capital of France?  document_retrieval
3     Show me documents on machine learning.     text_generation
4                          Tell me a riddle.  document_retrieval
..                                       ...                 ...
134        Search for news on electric cars.     text_generation
135                          Tell me a joke.     text_generation
136  Search for papers on quantum computing.  document_retrieval
137        Search for news on electric cars.  document_retrieval
138  Search for papers on quantum computing.  document_retrieval

[139 rows x 2 columns]


In [15]:
print(data)


                                       Query            Category
0               What is the meaning of life?  document_retrieval
1        Show me documents about blockchain.  document_retrieval
2             What is the capital of France?  document_retrieval
3     Show me documents on machine learning.     text_generation
4                          Tell me a riddle.  document_retrieval
..                                       ...                 ...
134        Search for news on electric cars.     text_generation
135                          Tell me a joke.     text_generation
136  Search for papers on quantum computing.  document_retrieval
137        Search for news on electric cars.  document_retrieval
138  Search for papers on quantum computing.  document_retrieval

[139 rows x 2 columns]


In [18]:
st.write("Data Columns:", data.columns)
st.write("Data Head:")
st.write(data.head())



In [20]:
expected_columns = ['query', 'category']
actual_columns = data.columns.tolist()
print(actual_columns)

['Query', 'Category']


In [21]:
st.write("Expected Columns:", expected_columns)
st.write("Actual Columns:", actual_columns)

In [22]:
# Preprocess data
def preprocess_data(data):
    # Rename columns to lowercase
    data.columns = [col.lower() for col in data.columns]
    if 'query' not in data.columns or 'category' not in data.columns:
        raise KeyError("Columns 'query' and 'category' are not found in the DataFrame.")
    return data[['query', 'category']]

In [23]:
data = preprocess_data(data)
st.write("Preprocessed Data:", data.head())

In [26]:
# Encode labels
label_encoder = LabelEncoder()
data['category'] = label_encoder.fit_transform(data['category'])

In [27]:
# Split data
X = data['query'].tolist()
y = data['category'].tolist()

In [28]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y)

# Tokenize data
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
def tokenize_data(data, tokenizer):
    return tokenizer(data, padding=True, truncation=True, max_length=512, return_tensors='pt')

X_train_tokenized = tokenize_data(X_train, tokenizer)
X_val_tokenized = tokenize_data(X_val, tokenizer)


In [31]:
# Define dataset class
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)


In [32]:
# Define training arguments and trainer
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred, average='weighted')
    precision = precision_score(y_true=labels, y_pred=pred, average='weighted')
    f1 = f1_score(y_true=labels, y_pred=pred, average='weighted')
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)


In [33]:
# Train the model
trainer.train()


<ipython-input-31-bfc874775a27>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Step,Training Loss
10,0.743600
20,0.704800


TrainOutput(global_step=21, training_loss=0.7238181006340754, metrics={'train_runtime': 76.2364, 'train_samples_per_second': 4.368, 'train_steps_per_second': 0.275, 'total_flos': 1882374601140.0, 'train_loss': 0.7238181006340754, 'epoch': 3.0})

In [34]:
# Evaluate the model
eval_results = trainer.evaluate()
st.write("Evaluation Results:", eval_results)


<ipython-input-31-bfc874775a27>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


2024-07-29 07:37:07.402 Session state does not function when running a script without `streamlit run`
